In [1]:
import torch
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2
import logging
from nowcasting.hko.dataloader import HKOIterator
from nowcasting.config import cfg
import numpy as np

In [2]:
from nowcasting.models.forecaster import Forecaster
from nowcasting.models.encoder import Encoder
from nowcasting.models.convLSTM import ConvLSTM
from collections import OrderedDict
from nowcasting.utils import rainfall_to_pixel

In [3]:
from nowcasting.models.loss import CrossEntropyLoss

In [4]:
S, B, C, H, W = 20, 2, 6, 24, 24
data = torch.randn(S, B, C, H, W).to(cfg.GLOBAL.DEVICE)
data.requires_grad = True
target = torch.randn(S, B, 1, H, W).to(cfg.GLOBAL.DEVICE)

In [5]:
criterion = CrossEntropyLoss().to(cfg.GLOBAL.DEVICE)

In [6]:
loss = criterion(data, target)

In [7]:
loss

tensor(2.1738, device='cuda:0', grad_fn=<MeanBackward1>)

In [8]:
loss.backward()

In [10]:
data.grad.size()

torch.Size([20, 2, 6, 24, 24])

In [11]:
weight = (torch.tensor([0, 1, 2, 5, 10, 30]) + 1).to(cfg.GLOBAL.DEVICE).float()

In [12]:
weight

tensor([ 1.,  2.,  3.,  6., 11., 31.], device='cuda:0')

In [13]:
criterion = CrossEntropyLoss(weight, 0.1).to(cfg.GLOBAL.DEVICE)

In [15]:
loss = criterion(data, target)
loss.backward()

In [104]:
target1 = target.permute((1, 2, 0, 3, 4)).squeeze(1)
class_index = torch.zeros_like(target1).long()
thresholds = [rainfall_to_pixel(ele) for ele in cfg.HKO.EVALUATION.THRESHOLDS]
for i, threshold in enumerate(thresholds, 1):
    class_index[target1 >= threshold] = i

In [105]:
real_prob = torch.zeros_like(data)

In [106]:
for i in range(S):
    for j in range(B):
        for k in range(H):
            for l in range(W):
                real_prob[i, j, class_index[j, i, k, l], k, l] = 1.0

In [113]:
real_prob.size()

torch.Size([20, 2, 6, 24, 24])

In [109]:
data.size()

torch.Size([20, 2, 6, 24, 24])

In [110]:
loss.backward()

In [112]:
data.grad.size()

torch.Size([20, 2, 6, 24, 24])

In [114]:
target.size()

torch.Size([2, 20, 24, 24])